## 循环神经网络

### 1 语言模型
假设序列$w_1, w_2, ..., w_T$的每个词是依次生成的，则
$$
P(w_1, ..., w_T) = \prod_{t=1}^T P(w_t | w_1, ..., w_{t-1}).
$$
基于$n-1$阶马尔可夫链，语言模型可改写为
$$
P(w_1, ..., w_T) \approx \prod_{t=1}^T P(w_t | w_{t-(n-1)}, ..., w_{t-1}),
$$
即当前词的出现仅和前面的$n-1$个词有关，这就是**$n$元语法**。

### 2 循环神经网络模型结构
循环神经网络并非刚性地记忆所有固定长度的序列，而是通过隐藏状态来存储之前时间步的信息。

在MLP中，设输入的小批量数据样本为$X \in \mathbb{R}^{n \times d}$，则隐藏层的输出为$H = \phi(XW_{xh} + b_h) \in \mathbb{R}^{n \times h}$，输出层的输出为$O = HW_{hq} + b_q \in \mathbb{R}^{n \times q}$，最后通过$softmax(O)$得到输出类别的概率分布。

在MLP的基础上，将上一时间步隐藏层的输出作为这一时间步隐藏层计算的输入，即

$$
H_t = \phi (X_t W_{xh} + H_{t-1}W_{hh} + b_h),
$$

通过引入新的权重参数将上一轮隐藏层的输出作为本轮隐藏层计算的依据之一。输出层的计算和MLP一致。

采用这种方式构建的循环神经网络的参数包含$W_{xh} \in \mathbb{R}^{d \times h}, W_{hh} \in \mathbb{R}^{h \times h}, b_h \in \mathbb{R}^{1 \times h}, W_{hq} \in \mathbb{R}^{h \times q}, b_q \in \mathbb{R}^{1 \times q}$。

![avatar](../resource/rnn.svg)

在时间步$t$，隐藏状态的计算可以看成是将输入$X_t$和前一时间步隐藏状态$H_{t-1}$连结后输入一个激活函数为$\phi$的全连接层。该全连接层的输出就是当前时间步的隐藏状态$H_t$且模型参数为$W_{xh}$和$W_{hh}$的连结，偏差为$b_h$。 

验证连接操作：

In [42]:
import torch

X, W_xh = torch.randn(3, 1), torch.randn(1, 4)
H, W_hh = torch.randn(3, 4), torch.randn(4, 4)
torch.matmul(X, W_xh) + torch.matmul(H, W_hh)

tensor([[ 0.3674, -1.2173, -2.8210,  0.8650],
        [ 0.4143, -0.0656, -0.1902,  0.8366],
        [-2.0265, -0.7649, -2.0427, -0.5145]])

In [2]:
torch.matmul(torch.cat((X, H), dim=1), torch.cat((W_xh, W_hh), dim=0))

tensor([[-1.5170, -1.2029, -0.8531,  1.7625],
        [-1.8118, -1.3473,  3.0991, -0.4491],
        [-0.5901, -0.8242,  2.1464,  0.1560]])

基于字符级循环神经网络来创建语言模型，即输入是一个字符，神经网络基于当前和过去的字符来预测下一个字符。在训练时，我们对每个时间步的输出层输出使用softmax运算，然后使用交叉熵损失函数来计算它与标签的误差。

![avatar](../resource/rnn-train.svg)

### 3 语言模型数据集
读取周杰伦专辑歌词：

In [3]:
import random
import zipfile

with zipfile.ZipFile('../data/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')
corpus_chars[:45]

'想要有直升机\n想要和你飞到宇宙去\n想要和你融化在一起\n融化在宇宙里\n我每天每天每天在想想想'

In [4]:
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[0:10000]

为每个字符构建索引：

In [5]:
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
print(vocab_size)

1027


In [6]:
corpus_indices = [char_to_idx[char] for char in corpus_chars]
sample = corpus_indices[:20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [65, 73, 1004, 694, 236, 462, 861, 65, 73, 246, 1012, 625, 428, 172, 715, 606, 861, 65, 73, 246]


### 4 时序数据的采样

**随机采样：**每次从数据里随机采样一个小批量。其中批量大小batch_size指每个小批量的样本数，num_steps为每个样本所包含的时间步数。 在随机采样中，每个样本是原始序列上任意截取的一段序列。相邻的两个随机小批量在原始序列上的位置不一定相毗邻。因此，我们无法用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态。在训练模型时，每次随机采样前都需要重新初始化隐藏状态。

In [7]:
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    # 一个example是一个包含了num_steps时间步的样本
    # 减1是因为输出的索引x是相应输入的索引y加1
    num_examples = (len(corpus_indices) - 1) // num_steps
    # batch_num是batch的个数
    batch_num = num_examples // batch_size
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)      # 打乱每个样本的顺序（单个样本内仍然是按照原本序列的顺序排列的）
    
    def _data(pos):
        return corpus_indices[pos: pos + num_steps]
    
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for i in range(batch_num):
        i = i * batch_size
        batch_indices = example_indices[i: i + batch_size]
        X = [_data(j * num_steps) for j in batch_indices]
        Y = [_data(j * num_steps + 1) for j in batch_indices]
        yield torch.tensor(X, dtype=torch.float32, device=device), torch.tensor(Y, dtype=torch.float32, device=device)

In [8]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY: ', Y, '\n')

X:  tensor([[ 0.,  1.,  2.,  3.,  4.,  5.],
        [ 6.,  7.,  8.,  9., 10., 11.]]) 
Y:  tensor([[ 1.,  2.,  3.,  4.,  5.,  6.],
        [ 7.,  8.,  9., 10., 11., 12.]]) 

X:  tensor([[18., 19., 20., 21., 22., 23.],
        [12., 13., 14., 15., 16., 17.]]) 
Y:  tensor([[19., 20., 21., 22., 23., 24.],
        [13., 14., 15., 16., 17., 18.]]) 



**相邻采样：**相邻的两个随机小批量在原始序列上的位置相毗邻。这时候，我们就可以用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态，从而使下一个小批量的输出也取决于当前小批量的输入，并如此循环下去。这对实现循环神经网络造成了两方面影响：一方面， 在训练模型时，我们只需在每一个迭代周期开始时初始化隐藏状态；另一方面，当多个相邻小批量通过传递隐藏状态串联起来时，模型参数的梯度计算将依赖所有串联起来的小批量序列。同一迭代周期中，随着迭代次数的增加，梯度的计算开销会越来越大。 为了使模型参数的梯度计算只依赖一次迭代读取的小批量序列，我们可以在每次读取小批量前将隐藏状态从计算图中分离出来。

In [9]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    corpus_indices = torch.tensor(corpus_indices, dtype=torch.float32, device=device)
    data_len = len(corpus_indices)        # 给定序列中字符的个数
    batch_len = data_len // batch_size
    indices = corpus_indices[0: batch_size * batch_len].view(batch_size, batch_len)
    batch_num = (batch_len - 1) // num_steps     # batch的个数
    for i in range(batch_num):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y

In [10]:
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY: ', Y, '\n')

X:  tensor([[ 0.,  1.,  2.,  3.,  4.,  5.],
        [15., 16., 17., 18., 19., 20.]]) 
Y:  tensor([[ 1.,  2.,  3.,  4.,  5.,  6.],
        [16., 17., 18., 19., 20., 21.]]) 

X:  tensor([[ 6.,  7.,  8.,  9., 10., 11.],
        [21., 22., 23., 24., 25., 26.]]) 
Y:  tensor([[ 7.,  8.,  9., 10., 11., 12.],
        [22., 23., 24., 25., 26., 27.]]) 



### 5 循环神经网络的从零开始实现

In [11]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import my_utils

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = my_utils.load_data_jay_lyrics()

对每个字符进行独热编码：

In [12]:
def one_hot(x, n_class, dtype=torch.float32):
    # input shape: (batch), output shape: (batch, n_class)
    x = x.long()
    res = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
    res.scatter_(1, x.view(-1, 1), 1)     # 将x作为索引，把1填充到res的dim=1的对应位置上
    return res

x = torch.tensor([0, 2])
one_hot(x, 10)

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])

作为net的输入：

每次小批量的形状为``(batch_size, num_steps)``，独热编码之后，小批量的输入为数个形状为``(batch_size, vocab_size)``的矩阵，矩阵个数为``num_steps``。

In [13]:
def to_onehot(X, n_class):
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]

X = torch.arange(10).view(2, 5)
inputs = to_onehot(X, n_class=10)
print(len(inputs), inputs[0].shape)

5 torch.Size([2, 10])


#### 5.1 初始化模型参数

In [14]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device, requires_grad=True))
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, requires_grad=True))
    return nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])

#### 5.2 定义模型
采用含隐藏状态的、具有单层隐藏层的MLP。

初始化隐藏状态：

In [15]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device),)

输入一个batch的数据应有的输出：

In [16]:
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [17]:
state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
print(len(outputs), outputs[0].shape, state_new[0].shape)

5 torch.Size([2, 1027]) torch.Size([2, 256])


#### 5.3 定义预测函数
基于前缀prefix（含有数个字符的字符串）来预测接下来的num_chars个字符。

In [18]:
def rnn_predict(prefix, num_chars, rnn, params, init_rnn_state, 
                num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
        (Y, state) = rnn(X, state, params)
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [19]:
rnn_predict('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size, device, idx_to_char, char_to_idx)

'分开市王及睛分蜜以晴谢之'

#### 5.4 裁剪梯度
循环神经网络中较容易出现梯度衰减或梯度爆炸。为了应对梯度爆炸，我们可以裁剪梯度（clip gradient），裁剪后的梯度的l2-norm不超过$\theta$:

$$
\min \bigg(\frac{\theta}{\| g \|}, 1\bigg) g
$$

In [20]:
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= theta / norm

#### 5.5 困惑度
使用困惑度（perplexity）评价语言模型的好坏。困惑度是对交叉熵损失函数做指数运算后得到的值。

* 最佳情况下，模型总是把标签类别的概率预测为1，此时困惑度为1；
* 最坏情况下，模型总是把标签类别的概率预测为0，此时困惑度为正无穷；
* 基线情况下，模型总是预测所有类别的概率都相同，此时困惑度为类别个数。

一个有效地模型的困惑度应在1和vocab_size之间。

#### 5.6 定义模型训练函数

In [21]:
def rnn_train_and_predict(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = data_iter_random
    else:
        data_iter_fn = data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:  
            # 否则需要使用detach函数从计算图分离隐藏状态, 这是为了
            # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                for s in state:
                    s.detach_()

            inputs = to_onehot(X, vocab_size)
            # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
            (outputs, state) = rnn(inputs, state, params)
            # 拼接之后形状为(num_steps * batch_size, vocab_size)
            outputs = torch.cat(outputs, dim=0)
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            # 使用交叉熵损失计算平均分类误差
            l = loss(outputs, y.long())

            # 梯度清0
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            my_utils.mgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', rnn_predict(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

使用随机采样的模型训练：

In [22]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [63]:
rnn_train_and_predict(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 68.140465, time 0.52 sec
 - 分开 我不要再想你 我有 我想你的可爱 我 想小的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让
 - 不分开 快颗的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱
epoch 100, perplexity 9.410330, time 0.58 sec
 - 分开 一只在一个棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快
 - 不分开吗 我不要 穿怎么 让我面的 是人我 别怪我 别你的三我有狂的可爱女人 坏坏的让我疯狂的可爱女人 坏
epoch 150, perplexity 2.843535, time 0.55 sec
 - 分开 一直在停留 白让它停抽屉 为所妙忆的手剩的钟 从反方向开始移动 回到我 别怪我 说你 是我去回乡 
 - 不分开吗 我爱你和 你我的外婆家 一起看着我打多得 我 想我你口睡车 我这想 你怎么抄我想 你说啊 是不是
epoch 200, perplexity 1.579061, time 0.52 sec
 - 分开 一直到美留 的话拽 小 店着我不吃不错不就  说一云看里日的河 从等方向开始移动 不是再这样打我妈
 - 不分开吗 然后将过去 慢慢温习 让我爱上你 那场悲剧 是你完美演出的一场戏 宁愿心碎哭泣 再狠狠忘记 你爱
epoch 250, perplexity 1.298882, time 0.61 sec
 - 分开 沙子了不哭 慢慢温 旧烧店 装属了明信片的铁盒里藏著一片玫瑰花瓣 对金葛爬满了雕花的门窗 夕阳斜斜
 - 不分开吗 我后了爸去 我后往再想  没有你烦我有多烦恼多难熬  穿过云层 我试著努力向你奔跑 爱才送到 你


使用相邻采样的模型训练：

In [64]:
rnn_train_and_predict(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 59.464333, time 0.49 sec
 - 分开 我想你这样 我不 我想 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我
 - 不分开 我想你的可爱 我的能 我不要的可爱 我的能 我不要这 快果我有 你你的让我疯狂的可爱女人 坏坏的让
epoch 100, perplexity 6.875133, time 0.50 sec
 - 分开 我说要 想情我的我不就 这你的美板在我像攻 我的你的可 在色莫我 全小了外屋 白静蜡烛 温暖了空屋
 - 不分开只 你是经离 是你村外不 我感多再想 我不能再想 我不 我不 我不要再想你 爱情走的太快就像龙卷风 
epoch 150, perplexity 1.996478, time 0.50 sec
 - 分开 我说到这样活 你有悄被我妈离得 我不了看不布许 你满么 干给了 我想就有手牵  它穿了其实 你话 
 - 不分开觉 你天经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 200, perplexity 1.273193, time 0.59 sec
 - 分开 问候我 谁是神枪手 巫师 他念念 有词的 对酋长下诅咒 还我骷髅头 这故事 告诉我 印地安的传说 
 - 不分开觉个 我不能 爱你我不起球 有话啊很医药是开  想去你在爸 对我 停地意义倒吧  通 你够再听幽每 
epoch 250, perplexity 1.154581, time 0.51 sec
 - 分开 问候我这多的可有一人 我都为伊 你来多定 我去没带宠 我的天空 是我爱很你在那植悲  是一场看剧花
 - 不分开觉个你我经妈了 那已的美在干起的河 在等待雨季来临变沼泽 灰狼啃食著水鹿的骨头 秃鹰盘旋死盯着腐肉 


### 6 循环神经网络的简洁实现

In [23]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = my_utils.load_data_jay_lyrics()

PyTorch中的nn模块提供了循环神经网络的实现。下面构造一个含单隐藏层、隐藏单元个数为256的循环神经网络层rnn_layer。

In [24]:
rnn_layer = nn.RNN(input_size=vocab_size, hidden_size=256)

这里rnn的输入为size=(num_steps, batch_size, covab_size)的tensor。

作为nn.RNN实例，在前向计算后会分别返回输出和隐藏状态h，其中**输出**指的是隐藏层在各个时间步上计算并输出的隐藏状态，它们通常作为后续输出层的输入。需要强调的是，该“输出”本身并不涉及输出层计算，形状为(num_steps, batch_size, hidden_size)。**隐藏状态**指的是隐藏层在最后时间步的隐藏状态（下图中的$(h_h, c_n)$）：当隐藏层有多层时，每一层的隐藏状态都会记录在该变量中；对于像长短期记忆（LSTM），隐藏状态是一个元组(h, c)，即hidden state和cell state。
![avatar](../resource/rnn_output.png)

In [43]:
num_steps = 35
batch_size = 2
state = None
X = torch.rand(num_steps, batch_size, vocab_size)
Y, state_new = rnn_layer(X, state)
print(Y.shape, len(state_new), state_new[0].shape)

torch.Size([35, 2, 256]) 1 torch.Size([2, 256])


#### 6.1 定义RNN模型

In [35]:
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.hidden_size = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1)
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size, vocab_size)
        self.state = None
    
    def forward(self, inputs, state):
        # input is of size (batch_size, num_steps)
        X = my_utils.to_onehot(inputs, self.vocab_size)
        Y, self.state = self.rnn(torch.stack(X), state)
        # change size into (num_steps * batch_size, num_hiddens)
        output = self.dense(Y.view(-1, Y.shape[-1]))
        return output, self.state

In [36]:
def rnn_predict_torch(prefix, num_chars, model, vocab_size, device, idx_to_char, char_to_idx):
    state = None
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]], device=device).view(1, 1)
        if state is not None:
            if isinstance(state, tuple):     # LSTM, state:(h, c)  
                state = (state[0].to(device), state[1].to(device))
            else:
                state = state.to(device)
        
        (Y, state) = model(X, state)
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [38]:
model = RNNModel(rnn_layer, vocab_size).to(device)
rnn_predict_torch('分开', 10, model, vocab_size, device, idx_to_char, char_to_idx)

'分开哼福野法的法爸法整爸'

In [39]:
def rnn_train_and_predict_torch(model, num_hiddens, vocab_size, device, 
                                corpus_indices, idx_to_char, char_to_idx, 
                                num_epochs, num_steps, lr, clipping_theta, 
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    state = None
    for epoch in range(num_epochs):
        l_sum, n, start = 0., 0, time.time()
        data_iter = my_utils.data_iter_consecutive(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if state is not None:
                if isinstance(state, tuple):
                    state = (state[0].detach(), state[1].detach())
                else:
                    state = state.detach()
            
            (output, state) = model(X, state)      # output: 形状为(num_steps * batch_size, vocab_size)
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            l = loss(output, y.long())
            
            optimizer.zero_grad()
            l.backward()
            my_utils.grad_clipping(model.parameters(), clipping_theta, device)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        
        try:
            perplexity = math.exp(l_sum / n)
        except OverflowError:
            perplexity = float('inf')
        
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, perplexity, time.time() - start))
            for prefix in prefixes:
                print(' -', rnn_predict_torch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))

In [41]:
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2 # 注意这里的学习率设置
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
rnn_train_and_predict_torch(model, num_hiddens, vocab_size, device,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 10.670113, time 0.39 sec
 - 分开 我不了口多 我  你已经我不多 我 我不要 我想就你 你你的我 你你的你我你多 我 我不能 我不多
 - 不分开 我不要再想 我不 我不 我不 我想 我不要再想 我不要再想 我不 我想 我想要你想要我不要 我想你
epoch 100, perplexity 1.287391, time 0.40 sec
 - 分开 我不到 爱情一定太极 一个会那 我想就这样打我妈妈 难道 说不会痛吗没我 别发抖 快给我抬起头 有
 - 不分开始 我不能痛 你是多 别你 没有你烦 我有多难熬  没有你在我有多难熬多烦恼  没有你烦 我有多烦恼
epoch 150, perplexity 1.066364, time 0.38 sec
 - 分开 我回到 身边缘 难离开始 快 我底开始我习造 我的可爱女人 想要就直升机 想要和你飞到宇宙去 想要
 - 不分开始 我不要痛 你你的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 漂亮的让我面红的可爱女人 温柔的让
epoch 200, perplexity 1.033532, time 0.38 sec
 - 分开 我回到 身边上 牵怎么一 像有话都做为我的心从在 是不要 想静句早已经 一起戏  后知后觉 我跟了
 - 不分开  回不了气 Wu  爱情来的太快就像龙卷风 离不开暴风圈来不及逃 我不能再想 我不能再想 我不 我
epoch 250, perplexity 1.021737, time 0.39 sec
 - 分开 你在我的多婆家 一一些风  有一碗热粥 配上几斤的牛肉 我说店小二 三两银够不够 景色入秋 漫天黄
 - 不分开  我不想气平常话不多 除非是乌鸦抢了它的窝 它在灌木丛旁邂逅 一只令它心仪的母斑鸠 印地安老斑鸠 
